In [1]:
# defining instance
import numpy as np
import time
# from KP_old import KnapsackProblem
from KP import KnapsackProblem

## Dynamic Programming for KP Partition Function 

**(August 26, 2022)**

We have a KP instance defined by a vector of values $\textbf{v} = (v_1, \ldots, v_N)$, a vector of weights $\textbf{w} = (w_1, \ldots, w_N)$ and a weight limit $W$. The partition function $Z_N(\beta \textbf{v}, \textbf{w}, W)$ (at inverse temperature $\beta$) for the KP obeys the recursive identity 

\begin{equation}
Z_N(\beta \textbf{v}, \textbf{w}, W)  = Z_{N-1}^{(N)}(\beta \textbf{v}, \textbf{w}, W) + e^{\beta v_N}Z^{(N)}_{N-1}(\beta \textbf{v}, \textbf{w}, W-w_N),
\label{eq:dynamic_prog}
\end{equation}

where $Z^{(N)}_{N-1}(\beta \textbf{v}, \textbf{w}, W)$ is the partition function for which the $N$th component is eliminated from both $\textbf{v}$ and $\textbf{w}$, and thus only $N-1$ items are under consideration. The above identity is tantamount to a dynamic programming solution to the KP partition function. In dynamic programming we build up the solution to a problem by writing said solution in terms of the stored solutions of sub-problems. Similarly the identity shows that we can build up the partition function for the main KP instance by writing it as the sum of partition functions for sub-instances. 

The partition function in turn allows us to determine the occupancy of object $j$ in the knapsack: We have

\begin{equation}
X_j  = 1 - \lim_{\beta\to\infty}\frac{Z_{N-1}^{(j)}(\beta\textbf{v}, \textbf{w}, W)}{Z_{N} (\beta\textbf{v}, \textbf{w}, W)},
\label{eq:XsolnSimp}
\end{equation}

where $Z^{(j)}_{N-1}(\beta \textbf{v}, \textbf{w}, W)$ is the partition function for which the $j$th component is eliminated from both $\textbf{v}$ and $\textbf{w}$. For numerical implementations, we generally cannot take $\beta\to \infty$ without overflow errors, so we will simply take $\beta$ to be large and finite (or $T$ to be small) after which we will threshold the result for $X_j$ to get a binary prediction.


### Knapsack Instance

In the following examples, we will use the item list, weights, values, and weight limits given as follows. These values are taken from the problem statement in [RossettaCode Knapsack: 0-1](https://rosettacode.org/wiki/Knapsack_problem/0-1)

In [2]:
items = (
    ("map", 9, 150), ("compass", 13, 35), ("water", 153, 200), ("sandwich", 50, 160),
    ("glucose", 15, 60), ("tin", 68, 45), ("banana", 27, 60), ("apple", 39, 40),
    ("cheese", 23, 30), ("beer", 52, 10), ("suntan cream", 11, 70), ("camera", 32, 30),
    ("t-shirt", 24, 15), ("trousers", 48, 10), ("umbrella", 73, 40),
    ("waterproof trousers", 42, 70), ("waterproof overclothes", 43, 75),
    ("note-case", 22, 80), ("sunglasses", 7, 20), ("towel", 18, 12),
    ("socks", 4, 50), ("book", 30, 10),
    )

# defining weight and value vectors and weight limit
weight_vec = np.array([item[1] for item in items])
value_vec = np.array([item[2] for item in items])
Wlimit = 400

# defining instance of problem
KP_camping = KnapsackProblem(weights = weight_vec, values = value_vec, limit = Wlimit)

**Dynamic Programming: Partition Function**
```
def exact_z_algorithm(self, rounded=True):
    """
    Computes the exact partition function for the KP 
    using dynamical programming and then uses this solution
    to write the solution to the KP.

    """

    # defining temperature as lowest as possible
    # according to python's overflow limit
    T = np.sum(self.values)/np.log(1.797693e308)

    # item number and weight limit
    N, W = len(self.weights), self.limit

    # defining empty partition function matrix
    Z = [[1 for x in range(W + 1)] for q in range(N + 1)]

    # implementing recursion identity 
    for i in range(1, N + 1):
        wt, val = self.weights[i-1], self.values[i-1]
        for w in range(1, W + 1):                
            if w < wt:
                Z[i][w] = Z[i-1][w]                   
            else:                     
                # partition function identity 
                Z[i][w] = Z[i-1][w] + np.exp(val/T)*Z[i-1][w-wt]         

    # computing occupancy vector
    occupancy = [0]*N
    w = W
    for j in range(N, 0, -1):
        # using X_j definition with simple half threshold
        was_added = 1 - Z[j-1][w]/Z[j][w] > 0.5

        if was_added:
            occupancy[j-1] = 1
            w -= self.weights[j-1]

    return occupancy 
```

In [3]:
# exact partition function algorithm
soln_exact_z = KP_camping.exact_z_algorithm()

**Dynamic Programming: Knapsack Maximum Values**
```
def knapsack01_dpV(self):
    '''
    Dynamical programming solution to knapsack problem
    Table elements are values
    '''

    # item number and weight limit
    N, W = len(self.weights), self.limit

    # empty value matrix
    V = [[0 for w in range(W + 1)] for j in range(N + 1)]

    # computing value matrix
    for j in range(1, N + 1):
        wt, val = self.weights[j-1], self.values[j-1]
        for w in range(1, W + 1):
            if w< wt:
                V[j][w] = V[j-1][w]
            else:
                V[j][w] = max(V[j-1][w],
                                  V[j-1][w-wt] + val)

    # computing occupancy vector
    occupancy = [0]*N
    w = W
    for j in range(N, 0, -1):
        was_added = V[j][w] != V[j-1][w]

        if was_added:
            occupancy[j-1] = 1
            w -= self.weights[j-1]

    return occupancy
```

In [4]:
# dynamical programming algorithm
soln_dpV = KP_camping.knapsack01_dpV()

#### Comparing Solutions for Example Instance

In [5]:
# start clock
start_clock0 = time.time()
print('Exact Z Solution')
for k in range(len(soln_exact_z)):
    if soln_exact_z[k] == 1:
        print(items[k][0])
print('Elapsed time: %.5f sec' % (time.time()-start_clock0))        

print()        
print('------------')
print()
# start clock
start_clock0 = time.time()
print('Dynamical Programming Solution')        
for k in range(len(soln_dpV)):
    if soln_dpV[k] == 1:
        print(items[k][0])  
print('Elapsed time: %.5f sec' % (time.time()-start_clock0))        

Exact Z Solution
map
compass
water
sandwich
glucose
banana
suntan cream
waterproof trousers
waterproof overclothes
note-case
sunglasses
socks
Elapsed time: 0.00108 sec

------------

Dynamical Programming Solution
map
compass
water
sandwich
glucose
banana
suntan cream
waterproof trousers
waterproof overclothes
note-case
sunglasses
socks
Elapsed time: 0.00034 sec
